# 1. d2l的例子

In [21]:
import torch
from torch import nn
from torch.nn import functional as F

class Inception(nn.Module):
    def __init__(self,in_channels,c1,c2,c3,c4):
        super(Inception,self).__init__()
        self.conv1 = nn.Conv2d(in_channels,c1,kernel_size = 1)
        self.conv2 = nn.Sequential(
                    nn.Conv2d(in_channels,c2[0],kernel_size = 1),nn.ReLU(),
                    nn.Conv2d(c2[0],c2[1],kernel_size=3,padding=1),nn.ReLU()
                )
        self.conv3 = nn.Sequential(
                    nn.Conv2d(in_channels,c3[0],kernel_size=1),nn.ReLU(),
                    nn.Conv2d(c3[0],c3[1],kernel_size=5, padding=2),nn.ReLU()
                )
        self.conv4 = nn.Sequential(
                    nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                    nn.Conv2d(in_channels,c4,kernel_size = 1),nn.ReLU(),
                )
    def forward(self,X):
        print("--X shape:",X.shape)
        p1 = self.conv1(X)
        p2 = self.conv2(X)
        p3 = self.conv3(X)
        p4 = self.conv4(X)

        return torch.cat((p1, p2, p3, p4), dim=1)


In [24]:
class GoogleNet(nn.Module):
    def __init__(self,in_channels,classes):
        super(GoogleNet,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels,out_channels=64,kernel_size=7,stride=2,padding=3),nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1),nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,padding=1),nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            Inception(192,c1=64,c2=[96,128],c3=[16,32],c4=32),
            # Inception(256,c1=128,c2=[128,192],c3=[32,96],c4=64),
            # nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
            # Inception(480,c1=192,c2=[96,208],c3=[16,48],c4=64),
            # Inception(512,c1=160,c2=[112,224],c3=[24,64],c4=64),
            # Inception(512,c1=128,c2=[128,256],c3=[24,64],c4=64),
            # Inception(512,c1=112,c2=[144,288],c3=[32,64],c4=64),
            # Inception(528,c1=256,c2=[160,320],c3=[32,128],c4=128),
            # nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
            # Inception(832,c1=256,c2=[160,320],c3=[32,128],c4=128),
            # Inception(832,c1=384,c2=[192,384],c3=[48,128],c4=128),
            # nn.AvgPool2d(kernel_size=7,stride=1),
            # nn.Dropout(p=0.4),
            # nn.Flatten(),
            # nn.Linear(1024,classes),
            # nn.Softmax(dim=1)
                )
    def forward(self,X:torch.tensor):
        for layer in self.model:
            X = layer(X)
            print(layer.__class__.__name__,'output shape:',X.shape)

In [25]:
X = torch.randn(size=(1,3,224,224))
net = GoogleNet(3,1000)
net(X)

Conv2d output shape: torch.Size([1, 64, 112, 112])
ReLU output shape: torch.Size([1, 64, 112, 112])
MaxPool2d output shape: torch.Size([1, 64, 55, 55])
Conv2d output shape: torch.Size([1, 64, 55, 55])
ReLU output shape: torch.Size([1, 64, 55, 55])
Conv2d output shape: torch.Size([1, 192, 55, 55])
ReLU output shape: torch.Size([1, 192, 55, 55])
MaxPool2d output shape: torch.Size([1, 192, 27, 27])
--X shape: torch.Size([1, 192, 27, 27])
Inception output shape: torch.Size([1, 256, 27, 27])


# 2.

In [ ]:
class Inception_Block_V1(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V1, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        # self.ln_in = int((300-kernel_size)/stride+1)
        for i in range(self.num_kernels):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=2 * i + 1, padding=i))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res
